In [30]:
output_path = '../Data/Intermediate_Files/'

# Import functions to clean up clinical data
from FM_Functions.Clinical_Data_CleanUp import *

# Call functions to merge, index and clean clinical data files
labels_0531         = clean_cog       (merge_index_0531())
labels_1031         = clean_cog       (merge_index_1031())
labels_aml05        = clean_aml05     (merge_index_aml05())
labels_beataml      = clean_beataml   (merge_index_beataml())
labels_amltcga      = clean_amltcga   (merge_index_amltcga())
labels_nordic_all   = clean_nordic_all(merge_index_nordic_all())
labels_mds_taml     = clean_mds_taml  (merge_index_mds_taml())
labels_all_graal    = clean_all_graal (merge_index_all_graal())
labels_target_all   = clean_target_all(merge_index_target_all())

# Combine all clinical data labels into one dataframe
labels_combined = pd.concat([labels_aml05, labels_beataml,
                        labels_0531, labels_amltcga, labels_1031,
                        labels_nordic_all, labels_mds_taml,
                        labels_all_graal,labels_target_all], axis=0, join='outer')

# Read df
df = pd.read_pickle(output_path + 'df_batch_corrected.pkl')

# Remove samples that are not in the methyl dataset
df_labels = labels_combined.loc[labels_combined.index.isin(df.index)].sort_index()

# Save the clinical data labels
df_labels.to_csv(output_path + 'clinical_data.csv')

print('The clinical data has been indexed and cleaned.\n\
Exclusion of samples may be applied depending on the analysis.')

The clinical data has been indexed and cleaned.
Exclusion of samples may be applied depending on the analysis.


In [38]:
# Split `Comment` column by `:` and keep only the last term
labels_1031['Comment2'] = labels_1031['Comment'].str.split("AML ").str[-1]

In [40]:
labels_1031[['Comment','Comment2']]

,Comment,Comment2
Sample_ID,,
202897270052_R02C01,WHO Classification (final: path then study ent...,with t(9;11)(p22;q23); MLLT3-MLL
202897270052_R03C01,WHO Classification (final: path then study ent...,with t(8;21)(q22;q22); RUNX1-RUNX1T1
202897270052_R05C01,WHO Classification (final: path then study ent...,with inv(16)(p13q22) or t(16;16)(p13;q22); CBF...
202915600109_R04C01,WHO Classification (final: path then study ent...,with inv(16)(p13q22) or t(16;16)(p13;q22); CBF...
202897270052_R06C01,WHO Classification (final: path then study ent...,with maturation
...,...,...
202915410140_R07C01,WHO Classification (final: path then study ent...,WHO Classification (final: path then study ent...
202905580052_R04C01,WHO Classification (final: path then study ent...,with t(8;21)(q22;q22); RUNX1-RUNX1T1
202908540101_R04C01,WHO Classification (final: path then study ent...,with inv(16)(p13q22) or t(16;16)(p13;q22); CBF...


In [42]:
labels_1031[['Gene Fusion.1', 'Comment', 'Primary Cytogenetic Code','Karyotype']]

,Gene Fusion.1,Comment,Primary Cytogenetic Code,Karyotype
Sample_ID,,,,
202897270052_R02C01,KMT2A-MLLT3,WHO Classification (final: path then study ent...,MLL,"46,XX,t(9;11)(p22;q23)[20]"
202897270052_R03C01,RUNX1-RUNX1T1,WHO Classification (final: path then study ent...,t(8;21),"45,X,-Y,t(8;21)(q22;q22)[13]/45,idem,add(7)(q3..."
202897270052_R05C01,CBFB-MYH11,WHO Classification (final: path then study ent...,inv(16),"46,XX,inv(16)(p13.1q22)[20]"
202915600109_R04C01,CBFB-MYH11,WHO Classification (final: path then study ent...,inv(16),"46,XX,inv(16)(p13.1q22)[20]"
202897270052_R06C01,None,WHO Classification (final: path then study ent...,Other,"46,XY,t(12;17)(q24.2;q11.1)[19]/46,XY[1]"
...,...,...,...,...
202915410140_R07C01,NUP98-NSD1,WHO Classification (final: path then study ent...,Other,"46,XY,del(2)(q34q37.3), cryp t(5;11)(q35;p15.4..."
202905580052_R04C01,RUNX1-RUNX1T1,WHO Classification (final: path then study ent...,t(8;21),"46,XX,t(8;21)(q22;q22)[2]/45,idem,-X[7]/46,ide..."
202908540101_R04C01,CBFB-MYH11,WHO Classification (final: path then study ent...,inv(16),"46,XX,inv(16)(p13.1q22)[19]/46,XX[1]"
